# Load Libraries

In [2]:
import os
import json
import math
import pandas as pd

from models import Model
from classes import Segment, Satelite, Customer
from drawingmap import DrawingMap

# Functions

## Load Customers

In [19]:
def load_customers(DEBUG = False) -> list[Customer]:
    customers = []
    df = pd.read_csv('../others/Levantamiento de Información/customers_20210119.csv')
    df = df[(df.category < 16000) & (df.cajas > 0)].reset_index().copy()
    for i in range(len(df)):
        new_customer = Customer(str(df.id_cliente[i]),
                                df.loc[i, 'lon'],
                                df.loc[i, 'lat'],
                                df.loc[i, 'cajas'],
                                df.loc[i, 'category']
                                )
        customers.append(new_customer)
    if DEBUG:
        print("-"*50)
        print("Count of customers: ", len(customers))
        print("First customer:")
        print(json.dumps(customers[0].__dict__,indent=2,default=str))
    return customers

## Load Satelites

In [20]:
def load_satelites(DEBUG = False) ->list[Satelite]:
    satelites = []
    df = pd.read_csv('../others/Levantamiento de Información/Informacion CD a Satelites.csv')
    df.fillna(0, inplace=True)
    for i in range(len(df)):
        capacity = {}
        numberVehiclesAvailable = {}
        costFixed = {}
        setSegmentCoverage = ['empty']
        newSatelite = Satelite(str(df.nombre[i]),
                               df.longitud[i],
                               df.latitud[i],
                               df.loc[i,'distance.value']/1000,
                               df.loc[i,'duration.value']/3600,
                               df.loc[i,'duration_in_traffic.value']/3600,
                               capacity,
                               numberVehiclesAvailable,
                               costFixed,
                               setSegmentCoverage,
                               )
        satelites.append(newSatelite)
    if DEBUG:
        print("-"*50)
        print("Count of SATELITES: ", len(satelites))
        print("First Satelite:")
        print(json.dumps(satelites[0].__dict__,indent=2,default=str))
    return satelites

## Load Matrixes: Distance and Durations with and without traffic

In [21]:
def distance(s: Satelite, c: Customer) -> float:
     return int(math.sqrt(math.pow(s.lat - c.lat, 2) + math.pow(s.lon - c.lon, 2))*1000)

In [22]:
def calculate_distance(satelites: list[Satelite], customers: list[Customer], DEBUG = False) -> dict[(str,str), float]:
    distance_matrix = {}
    distance_matrix = dict(
        [((s.id, c.id), distance(s,c)) for s in satelites for c in customers]
    )
    if DEBUG:
        print("-"*50)
        print("Count - distance: ", len(distance_matrix))
    return distance_matrix

# Drawing Map

In [28]:
s = load_satelites()
c = load_customers()
distance_matrix = calculate_distance(s,c)

In [41]:
location_la_paz = [-16.501457, -68.149887]
drawer = DrawingMap(location_la_paz)
map = drawer.viewMap()
map

In [42]:
hue = drawer.setHue(13000,13100)
drawer.addCustomers(c,label='category', radius=2, fill=False)
map = drawer.viewMap()
map